# Objectifs 

Prise en main des fonctionnalité de `scikit-learn` suivants.
- Normalisation (`StandardScaler`, `MinMaxScaler`, `MaxAbsScaler`, `RobustScaler`)
- Transformation de données catégorielle en quantitative (`LabelEncoder`, `OneHotEncoder`) 
- Gestion des données manquantes (`SimpleImputer`, `KNNImputer`) 
- Composition de différentes étapes (`Pipeline`, `ColumnTransformer`, `FeatureUnion`)

Mise en oeuvre sur le tableau de données [immobilier](https://raw.githubusercontent.com/VPerrollaz/immobilier/master/donnees/data.tsv) pour obtenir un prédicteur de prix le plus fiable possible.

1. Charger les données dans un dataframe pandas. (on pourra regarder aussi le module `pathlib` pour explorer les fichiers de manière robuste)
2. Explorer les données via pandas : quelles sont les features, y-a-t il beaucoup de NaN, quelles sont les variables importantes, quelles sont leurs types, quelles sont les plages de variations.
3. En déduire des pipelines sklearn (avec preprocessing et algorithme) avant de choisir le meilleur par crossvalidation.

# Chargement avec pandas

In [1]:
import pandas as pd

In [2]:
donnees = pd.read_csv("https://raw.githubusercontent.com/VPerrollaz/immobilier/master/donnees/data.tsv", sep="\t")

In [3]:
donnees.describe()

,Neuf,Surface,Pieces,Prix
count,1647.000000,1639.000000,1646.000000,1.646000e+03
mean,0.138434,97.953392,4.047388,2.621814e+05
std,0.345459,82.332672,2.436838,2.666522e+05
min,0.000000,11.960000,1.000000,2.980000e+04
25%,0.000000,57.205000,3.000000,1.225000e+05
50%,0.000000,73.190000,3.000000,1.796735e+05
75%,0.000000,108.000000,5.000000,2.941500e+05
max,1.000000,1400.000000,25.000000,5.596080e+06


In [4]:
donnees.head()

,Id,Genre,Neuf,Surface,Pieces,Quartier,Prix
0,annonce-138905473-376235,Appartement,0,90.00,3.0,cathédrale,374400.0
1,annonce-140620177-376235,Appartement,0,146.27,5.0,sud,499200.0
2,annonce-140620179-376235,Appartement,0,110.00,5.0,prébendes,499200.0
3,annonce-133494153-376235,Maison,0,132.00,6.0,prébendes,508000.0
4,annonce-137425993-376235,Maison,0,185.00,7.0,strasbourg,676000.0


**ATTENTION**
Pour les versions et les identifiants même si on n'utilise que des chiffres on utilise le type `str` pour éviter les erreurs d'arrondis si jamais ils sont interprétés comme `float`.

In [5]:
donnees.dtypes

Id           object
Genre        object
Neuf          int64
Surface     float64
Pieces      float64
Quartier     object
Prix        float64
dtype: object

# Premières modifications et Exploration avec Pandas

**Modifications**

- Supprimer les éventuels doublons
- Attribuer les bons types aux colonnes.

**Remarque** 

- L'API de `pandas` est gigantesque et peu "*découvrable*" on pourra commencer par se concentrer sur la [cheatsheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) 
- On pourrait utiliser la librairie [`pyjanitor`](https://pyjanitor.readthedocs.io/)

In [6]:
donnees = donnees.convert_dtypes()
donnees.dtypes

Id           string
Genre        string
Neuf          Int64
Surface     float64
Pieces        Int64
Quartier     string
Prix          Int64
dtype: object

On pourrait vouloir forcer la conversion de la colonne `Surface` en `Int`.

In [7]:
#donnees.Surface = donnees.Surface.apply(int)

**ATTENTION** en python `NaN` est forcément un flottant...

**Remarque** on aurait pu utiliser une méthode spécifique pour les types : `astype`.

In [8]:
donnees = donnees.drop_duplicates().drop(axis=1, labels="Id")
donnees.columns

Index(['Genre', 'Neuf', 'Surface', 'Pieces', 'Quartier', 'Prix'], dtype='object')

# Numériser les colonnes

In [9]:
donnees.dtypes

Genre        string
Neuf          Int64
Surface     float64
Pieces        Int64
Quartier     string
Prix          Int64
dtype: object

In [12]:
donnees.Genre.value_counts()

Appartement    1241
Maison          392
Name: Genre, dtype: Int64

In [13]:
def filtre(genre):
    if genre == "Appartement":
        return 0
    elif genre == "Maison":
        return 1
donnees.Genre = donnees.Genre.apply(filtre)
donnees.dtypes

Genre         int64
Neuf          Int64
Surface     float64
Pieces        Int64
Quartier     string
Prix          Int64
dtype: object

# Regarder les valeurs manquantes 

In [14]:
donnees.count()

Genre       1633
Neuf        1633
Surface     1625
Pieces      1632
Quartier     948
Prix        1632
dtype: int64

1. On va dégager les annonces problématiques pour les variables `Surface`, `Pieces` et `Prix` car ça ne modifie quasiment pas la taille de l'échantillon.
2. Pour `Quartier` il va falloir être plus prudent. On pourra supprimer les lignes non remplies (mais ça diminue beaucoup l'échantillon), on pourra remplir avec un quartier artificiel, on pourra une fois appliqué `OneHotEncoder` remplir numériquement suivant la probabilité d'être dans un quartier, on peut finalement éliminer la colonne `Quartier`. Ces choix constitueront une étape pour lesquelles on sélectionnera par cross-validation.

In [15]:
donnees = donnees.dropna(axis=0, subset=["Surface", "Pieces", "Prix"])
donnees.count()

Genre       1625
Neuf        1625
Surface     1625
Pieces      1625
Quartier     944
Prix        1625
dtype: int64

In [16]:
donnees_sans_quartier = donnees.drop(axis=1, labels="Quartier")
donnees_sans_quartier.columns

Index(['Genre', 'Neuf', 'Surface', 'Pieces', 'Prix'], dtype='object')

In [17]:
donnees_avec_quartier = donnees.dropna(axis=0)
donnees_avec_quartier.count()

Genre       944
Neuf        944
Surface     944
Pieces      944
Quartier    944
Prix        944
dtype: int64

In [18]:
donnees_quartier_artificiel = donnees.fillna(value={"Quartier": "Fictif"})
print(donnees_quartier_artificiel.count())
donnees_quartier_artificiel.Quartier.value_counts()

Genre       1625
Neuf        1625
Surface     1625
Pieces      1625
Quartier    1625
Prix        1625
dtype: int64


Fictif           681
nord             180
centre           161
cher              90
sud               90
halles            63
prébendes         61
gare              51
velpeau           36
cathédrale        35
beaujardin        28
portes            19
2 lions           18
radegonde         18
febvotte          15
strasbourg        14
tranchée          12
montjoyeux        12
paul bert         10
heurteloup         9
eloi               5
mairie             4
conservatoire      4
cluzel             3
fontaines          3
béranger           2
rotonde            1
Name: Quartier, dtype: Int64

In [19]:
donnees_quartier_probabiliste = ...

# Identifier les variables numériques importantes.